# Medical Q&A Chatbot Development Notebook

In [11]:
!pip install --upgrade typing_extensions

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.2.6 requires typing-extensions<4.6.0,>=3.7.4.1, but you have typing-extensions 4.13.1 which is incompatible.



  Using cached typing_extensions-4.13.1-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.13.1-py3-none-any.whl (45 kB)
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0


In [12]:
!pip install -r requirements.txt

  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached pydantic-1.10.7-cp310-cp310-win_amd64.whl.metadata (146 kB)
  Using cached spacy-3.7.2-cp310-cp310-win_amd64.whl.metadata (26 kB)
  Using cached scispacy-0.5.0-py3-none-any.whl.metadata (15 kB)
  Using cached streamlit-1.27.0-py2.py3-none-any.whl.metadata (8.0 kB)
  Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached thinc-8.2.5-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached weasel-0.3.4-py3-none-any.whl.metadata (4.7 kB)
INFO: pip is looking at multiple versions of scispacy to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    The user requested spacy==3.7.2
    scispacy 0.5.0 depends on spacy<3.3.0 and >=3.2.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict



ERROR: Cannot install -r requirements.txt (line 4) and spacy==3.7.2 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [13]:
!pip install 'pybind11<2.6.2'

The system cannot find the file specified.


In [14]:
!pip install scispacy

  Using cached scispacy-0.5.5-py3-none-any.whl.metadata (18 kB)
  Using cached spacy-3.7.5-cp310-cp310-win_amd64.whl.metadata (27 kB)
  Using cached scipy-1.15.2-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached conllu-6.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.6.1-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl.metadata (6.1 kB)
  Using cached nmslib-2.1.1.tar.gz (188 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pybind11-2.6.1-py2.py3-none-any.whl.metadata (8.7 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached thinc-8.2.5-cp310-cp310-win_amd64.whl.metadata (15 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached numpy-1.26.4-cp310-cp310-win_amd6

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1197 lines of output]
      Dependence list: ['pybind11<2.6.2', 'psutil', "numpy>=1.10.0,<1.17 ; python_version=='2.7'", "numpy>=1.10.0 ; python_version>='3.5'"]
      C:\Users\dubey\anaconda3\envs\chatbot_env3\lib\site-packages\setuptools\dist.py:493: SetuptoolsDeprecationWarning: Invalid dash-separated options
      !!
      
              ********************************************************************************
              Usage of dash-separated 'description-file' will not be supported in future
              versions. Please use the underscore name 'description_file' instead.
      
              This deprecation is overdue, please update your project and remove deprecated
              calls to avoid build errors in the future.
      
              See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
              *

In [15]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz (120.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.1
    Uninstalling typing_extensions-4.13.1:
      Successfully uninstalled typing_extensions-4.13.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.35.0 requires typing_extensions>=4.6; python_version < "3.12", but you have typing-extensions 4.5.0 which is incompatible.


In [16]:
import scispacy
import en_ner_bc5cdr_md

nlp = en_ner_bc5cdr_md.load()
print(nlp.pipe_names)  # Should show ['ner']

ModuleNotFoundError: No module named 'scispacy'

In [ ]:
# 3. Import Libraries
import os
import numpy as np
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from tqdm import tqdm

In [ ]:
!pip install lxml tqdm

In [ ]:
# 3. Data Loading with proper XML structure handling
data_dir = "MedQuAD"
qa_pairs = []

# Walk through all directories and files with progress bar
for root, _, files in tqdm(os.walk(data_dir), desc="Processing files"):
    for filename in files:
        if filename.endswith('.xml'):
            file_path = os.path.join(root, filename)
            try:
                # Parse XML file
                tree = ET.parse(file_path)
                root_elem = tree.getroot()
                
                # Find all QA pairs in the XML structure
                qa_section = root_elem.find('QAPairs')
                if qa_section is not None:
                    for qa in qa_section.findall('QAPair'):
                        question = qa.find('Question')
                        answer = qa.find('Answer')
                        
                        if question is not None and answer is not None:
                            q_text = question.text.strip() if question.text else ""
                            a_text = answer.text.strip() if answer.text else ""
                            
                            if q_text and a_text:  # Only add if both exist
                                qa_pairs.append({
                                    'question': q_text,
                                    'answer': a_text
                                })
            except Exception as e:
                print(f"\nError in {file_path}: {str(e)}")
                continue

# Create DataFrame and verify
df = pd.DataFrame(qa_pairs)
print(f"\nLoaded {len(df)} QA pairs")

if not df.empty:
    print("\nFirst QA pair:")
    print(f"Question: {df.iloc[0]['question']}")
    print(f"Answer: {df.iloc[0]['answer'][:100]}...")  # Show first 100 chars
else:
    print("\nNo QA pairs loaded. Check XML structure and file paths.")  

In [ ]:
# 4. Text Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Handle missing values
    if not isinstance(text, str):
        return ''
    
    # Lowercase
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Tokenization and lemmatization
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to questions
df['processed_question'] = df['question'].apply(preprocess)


In [ ]:
# 5. TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize and fit TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['processed_question'])

In [ ]:
def evaluate_performance(df, test_fraction=0.2):
    # Split data into test indices
    test_indices = df.sample(frac=test_fraction, random_state=42).index
    answer_groups = df.groupby('answer').apply(lambda x: x.index.tolist())
    
    # Initialize metrics
    accuracy = 0.0
    mrr = 0.0
    total = len(test_indices)
    
    # Precompute TF-IDF matrix for all questions
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['processed_question'])
    
    for idx in tqdm(test_indices, desc="Evaluating Metrics"):
        test_question = df.loc[idx, 'question']
        test_answer = df.loc[idx, 'answer']
        
        # Preprocess and vectorize test question
        processed_query = preprocess(test_question)
        query_vec = vectorizer.transform([processed_query])
        
        # Calculate similarities while excluding current test question
        similarities = cosine_similarity(query_vec, X)
        similarities[0, idx] = -1  # Exclude self
        
        # Get most similar index
        most_similar_idx = similarities.argmax()
        retrieved_answer = df.loc[most_similar_idx, 'answer']
        
        # Update accuracy
        accuracy += 1 if retrieved_answer == test_answer else 0
        
        # Calculate MRR
        correct_indices = [i for i in answer_groups.get(test_answer, []) if i != idx]
        if correct_indices:
            sorted_indices = np.argsort(similarities[0])[::-1]
            for rank, sorted_idx in enumerate(sorted_indices, 1):
                if sorted_idx in correct_indices:
                    mrr += 1.0 / rank
                    break
    
    # Calculate final metrics
    accuracy /= total
    mrr /= total
    
    print(f"\nPerformance Metrics:")
    print(f"Top-1 Accuracy: {accuracy:.2%}")
    print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")

# Run evaluation
evaluate_performance(df)

In [ ]:
# 6. Answer Retrieval Function
def get_most_similar_answer(query):
    # Preprocess the query
    processed_query = preprocess(query)
    # Vectorize the query
    query_vec = vectorizer.transform([processed_query])
    # Calculate cosine similarities
    similarities = cosine_similarity(query_vec, X_tfidf)
    # Get index of most similar question
    most_similar_idx = similarities.argmax()
    # Return corresponding answer
    return df.iloc[most_similar_idx]['answer']


In [ ]:
# 7. Test the System
test_question = "What are the symptoms of diabetes?"
print(f"\nTest Question: {test_question}")
print("Answer:", get_most_similar_answer(test_question))


In [ ]:
# 8. Enhanced Medical Entity Recognition
import spacy
nlp = spacy.load("en_core_web_sm")

# Medical entity labels to look for (based on common medical terminology)
MEDICAL_LABELS = {
    'SYMPTOMS': ['pain', 'fever', 'headache', 'nausea', 'rash'],  # Add more symptoms
    'DISEASES': ['diabetes', 'cancer', 'leukemia', 'hypertension'],
    'TREATMENTS': ['chemotherapy', 'radiation', 'surgery', 'antibiotics']
}

def extract_medical_entities(text):
    doc = nlp(text)
    entities = []
    
    # Check for medical terms using both NER and keyword matching
    for ent in doc.ents:
        if ent.label_ in ['CHEMICAL', 'DISEASE']:  # spaCy's limited medical labels
            entities.append((ent.text, ent.label_))
    
    # Additional keyword-based matching
    for token in doc:
        text_lower = token.text.lower()
        for label, keywords in MEDICAL_LABELS.items():
            if text_lower in keywords:
                entities.append((token.text, label))
    
    # Remove duplicates
    return list(set(entities))

In [ ]:
# Test Enhanced Entity Recognition
test_text = "I have headache and fever for three days"
print("\nEnhanced Entity Recognition Test:")
print(f"Text: {test_text}")
print("Entities:", extract_medical_entities(test_text))

In [ ]:
!pip install streamlit

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

In [ ]:
!streamlit run med_chatbot.py